In [2]:
using Laplacians

objc[49484]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[49484]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[49484]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[49484]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [3]:
set_bigfloat_precision(1024)

1024

In [162]:
u = Array{Int64,1}(0)
v = Array{Int64,1}(0)
w = Array{Float64,1}(0)

n = 1000
for i in 1:(n-1)
    push!(u, i)
    push!(v, i+1)
    push!(w, 1.0)
    
    push!(u, i+1)
    push!(v, i)
    push!(w, 1.0)
end

valpool = rand() * exp(20 * rand(n));

for i in 1:(n-2)
    if i % 3 == 1
        extra = extra + 1
        
        val = valpool[i]
#        println(val)
        
        push!(u, i)
        push!(v, i+2)
        push!(w, val)
        
        push!(u, i+2)
        push!(v, i)
        push!(w, val)
    end
end

a = sparse(u,v,w); la = lap(a); n = a.n;

utree = collect(1:(n-1))
append!(utree, collect(2:n))

vtree = collect(2:n)
append!(vtree, collect(1:(n-1)))

wtree = ones(2 * n - 2)
tree = lap(sparse(utree,vtree,wtree));

In [163]:
full(la);

In [164]:
println(minimum(la.nzval))
vmax = -1000000
for v in la.nzval
    if v < -2
        vmax = max(v,vmax)
    end
end
println(vmax)

-1.1419260925695679e8
-2.2381948559787426


In [165]:
#eig(pinv(full(tree)) * la);

In [166]:
# n = 1250;
# a = chimera(n,44); la = lap(a);
x = rand(n);
b = la * x; b = b - mean(b);

In [167]:
srand(1)
# tree = lap(akpw(a)); 
cf = cholfact(tree,shift=1e-15);
p = cf[:p]
invp = invperm(p)
L = sparse(cf[:L])
U = L';

In [168]:
F = function(b)
    return cf \ (b - mean(b))
end

(anonymous function)

In [169]:
@time f = pcg(la, b, F, tol=1e-10, verbose=true);

Working on the 10th iteration
Working on the 20th iteration
Working on the 30th iteration
Working on the 40th iteration
Working on the 50th iteration
Working on the 60th iteration
Working on the 70th iteration
Working on the 80th iteration
Working on the 90th iteration
Working on the 100th iteration
Working on the 110th iteration
Working on the 120th iteration
Working on the 130th iteration
Working on the 140th iteration
Working on the 150th iteration
Working on the 160th iteration
Working on the 170th iteration
Working on the 180th iteration
Working on the 190th iteration
Working on the 200th iteration
Working on the 210th iteration
Working on the 220th iteration
Working on the 230th iteration
Working on the 240th iteration
Working on the 250th iteration
Working on the 260th iteration
Working on the 270th iteration
Working on the 280th iteration
Working on the 290th iteration
Working on the 300th iteration
Working on the 310th iteration
Working on the 320th iteration
Working on the 33

In [170]:
function to1024{Tv,Ti}(mat::SparseMatrixCSC{Tv,Ti})
    u,v,w = findnz(mat)
    w1024 = Array{BigFloat,1}(0)
    for val in w
        push!(w1024, BigFloat(val))
    end
    return sparse(u,v,w1024)
end

function to1024{Tv}(v::Array{Tv,1})
    res = Array{BigFloat,1}(0)
    for val in v
        push!(res, BigFloat(val))
    end
    return res
end

to1024 (generic function with 2 methods)

In [171]:
la1024 = to1024(la)
U1024 = UpperTriangular(to1024(U))
b1024 = to1024(b);

In [172]:
F1024 = function(b::Array{BigFloat,1})
    res = (copy(b) - mean(b))[p]

    res = U1024' \ res
    res = U1024 \ res

    return res[invp]
end

(anonymous function)

In [173]:
@time f = pcg(la1024, b1024, F1024, tol=1e-10, verbose=true);

Working on the 10th iteration
Working on the 20th iteration
Working on the 30th iteration
Working on the 40th iteration
Working on the 50th iteration
Working on the 60th iteration
Working on the 70th iteration
Working on the 80th iteration
Working on the 90th iteration
Working on the 100th iteration
Working on the 110th iteration
Working on the 120th iteration
Working on the 130th iteration
Working on the 140th iteration
Working on the 150th iteration
Working on the 160th iteration
Working on the 170th iteration
Working on the 180th iteration
Working on the 190th iteration
Working on the 200th iteration
Working on the 210th iteration
Working on the 220th iteration
Working on the 230th iteration
Working on the 240th iteration
Working on the 250th iteration
Working on the 260th iteration
Working on the 270th iteration
Working on the 280th iteration
Working on the 290th iteration
Working on the 300th iteration
Working on the 310th iteration
Working on the 320th iteration
Working on the 33